In [2]:
import pandas as pd
import numpy as np
import os.path
from arabert.arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score


In [3]:
#from transformers import BertTokenizer, TFBertModel

In [4]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer,AutoModel
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import csv
import optuna

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [8]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [15]:
path= r"C:\Users\96654\zatca_tweets22.csv"
df = pd.read_csv(path,header=0)
df = df.rename(columns={'content': 'text'})
#df['sentiment'] = np.random.randint(0, 3, df.shape[0])
model_name = "bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False,replace_urls_emails_mentions=True)

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
Q_COLUMN= 'question'
df = df[['text']]
#df = df[['text', 'sentiment']]
df.columns = [DATA_COLUMN]
#df.columns = [DATA_COLUMN, LABEL_COLUMN]

df['orginal'] = df[DATA_COLUMN]
df[DATA_COLUMN] = df[DATA_COLUMN].apply(arabert_prep.preprocess)
#df[LABEL_COLUMN] = df[LABEL_COLUMN].apply(lambda x: label_map[x])


test_AJGT = df




dev_df = pd.DataFrame({
    'id':range(len(test_AJGT)),
   # 'label':test_AJGT["label"],
    'text': test_AJGT["text"].replace(r'\n', ' ', regex=True),

})


In [16]:
len(df)

4698

In [17]:
df .columns

Index(['text', 'orginal'], dtype='object')

In [38]:
dev_df

,id,text
0,0,[مستخدم] [مستخدم] فارق لا ادعس على وجهك
1,1,التأثير على التاجر او المطور لانه سوف تزيد عليهم تكاليف ضريبة التصرفات العقارية ، و ضريبة الأراضي البيضاء و تأمين المباني السكنية .
2,2,عاجل | تصل ل 4 آلاف جنيه . . هذه قيمة ضريبة # التصرفات _ العقارية المستحقة على المواطنين . . ومقسمه ل 4 شرائح [رابط] [رابط]
3,3,احسب ضريبة التصرفات العقارية ل « بيتك أو أرضك » [رابط] [رابط]
4,4,بعد نفي شائعتها اليوم . . ماهي ضريبة التصرفات العقارية ؟ [رابط]
5,5,مصلحة الضرائب توضح طريقة فرض ضريبة التصرفات العقارية [رابط]
6,6,تتحدد وفقا لقيمة الصرف . . الضرائب تعلن طريقة حساب ضريبة التصرفات العقارية # التصرفات _ العقارية # قانون _ التصرفات _ العقارية # مصلحة _ الضرائب [رابط]
7,7,بعد نفي شائعتها اليوم . . احسب ضريبة التصرفات العقارية ل « بيتك أو أرضك » [رابط]
8,8,استثناءات تطبيق ضريبة التصرفات العقارية [رابط]
9,9,احسب ضريبة التصرفات العقارية ل « بيتك أو أرضك » . . عاجل [رابط] [رابط]


In [19]:
len(test_AJGT)

4698

In [20]:
df.shape

(4698, 2)

In [39]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [44]:
class BERTDataset(Dataset):
    def __init__(self, text, model_name, max_len):
      super(BERTDataset).__init__()
      self.text = text
      #self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = BertTokenizer.from_pretrained(model_name,from_pt=True)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [23]:
model_name="bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)

'ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري'

In [87]:
#from transformers import TFBertForMaskedLM

In [88]:
#get 'https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/tf1_model.tar.gz'

In [20]:
#test2 = test2.rename(columns={'content': 'text'})

In [45]:
max_len=64

In [46]:
test_dataset = BERTDataset(dev_df['text'].to_list(),model_name , max_len)

In [47]:
len(test_dataset)

4698

In [48]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
save_obj(label_map,'label_map')

In [52]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 32
training_args.per_device_eval_batch_size = 32
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 5




In [49]:
import tensorflow as tf

In [53]:
model=AutoModelForSequenceClassification.from_pretrained(r'C:\Users\96654\twee3\twitter_large')
trainer2 = Trainer(model=model,args = training_args)

C:\Users\96654\anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [84]:
test_dataset

In [54]:
result1= trainer2.predict(test_dataset)

In [55]:
result1

PredictionOutput(predictions=array([[ 0.4219983 ,  2.0677414 , -2.5789258 ],
       [ 1.4208467 ,  0.94053495, -1.5624692 ],
       [-1.12919   ,  2.5614834 , -1.2656647 ],
       ...,
       [ 0.72816026,  1.0042107 , -0.9265068 ],
       [ 2.155209  ,  0.47324818, -2.27634   ],
       [ 1.4475701 ,  1.2011757 , -1.9737192 ]], dtype=float32), label_ids=None, metrics={})

In [56]:
pred=result1.predictions

In [57]:
pred

array([[ 0.4219983 ,  2.0677414 , -2.5789258 ],
       [ 1.4208467 ,  0.94053495, -1.5624692 ],
       [-1.12919   ,  2.5614834 , -1.2656647 ],
       ...,
       [ 0.72816026,  1.0042107 , -0.9265068 ],
       [ 2.155209  ,  0.47324818, -2.27634   ],
       [ 1.4475701 ,  1.2011757 , -1.9737192 ]], dtype=float32)

In [66]:
df_pred = pd.DataFrame(pred)

In [68]:
df_pred

,0,1,2
0,0.421998,2.067741,-2.578926
1,1.420847,0.940535,-1.562469
2,-1.129190,2.561483,-1.265665
3,-1.551050,3.233905,-1.656919
4,-1.194193,3.262587,-1.843557
5,-1.475289,3.375949,-1.641937
6,-1.442079,2.809301,-1.206295
7,-1.131092,3.126127,-1.555888
8,-1.642793,3.685209,-1.804660
9,-1.546144,3.029299,-1.436554


In [58]:

final_pred=np.argmax(result1.predictions, axis=1)

final_result=[list(label_map.keys())[list(label_map.values()).index(x)] for x in final_pred]
true_text=dev_df['text'].to_list()




In [59]:
final_pred

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [71]:
sentimentResults = pd.DataFrame(
    {'text': true_text,
     'sentiment_BERT': final_result
    })
#sentimentResults.to_excel('BERT_tweets_RESULTS25_II.xlsx')

In [61]:
sentimentResults

,text,sentiment_BERT
0,[مستخدم] [مستخدم] فارق لا ادعس على وجهك,1
1,التأثير على التاجر او المطور لانه سوف تزيد عليهم تكاليف ضريبة التصرفات العقارية ، و ضريبة الأراضي البيضاء و تأمين المباني السكنية .,0
2,عاجل | تصل ل 4 آلاف جنيه . . هذه قيمة ضريبة # التصرفات _ العقارية المستحقة على المواطنين . . ومقسمه ل 4 شرائح [رابط] [رابط],1
3,احسب ضريبة التصرفات العقارية ل « بيتك أو أرضك » [رابط] [رابط],1
4,بعد نفي شائعتها اليوم . . ماهي ضريبة التصرفات العقارية ؟ [رابط],1
5,مصلحة الضرائب توضح طريقة فرض ضريبة التصرفات العقارية [رابط],1
6,تتحدد وفقا لقيمة الصرف . . الضرائب تعلن طريقة حساب ضريبة التصرفات العقارية # التصرفات _ العقارية # قانون _ التصرفات _ العقارية # مصلحة _ الضرائب [رابط],1
7,بعد نفي شائعتها اليوم . . احسب ضريبة التصرفات العقارية ل « بيتك أو أرضك » [رابط],1
8,استثناءات تطبيق ضريبة التصرفات العقارية [رابط],1
9,احسب ضريبة التصرفات العقارية ل « بيتك أو أرضك » . . عاجل [رابط] [رابط],1


In [62]:
path= r"C:\Users\96654\zatca_tweets22.csv"
df2 = pd.read_csv(path,header=0)

In [63]:
final_df = pd.merge(df2,sentimentResults,how="inner" ,left_index=True, right_index=True)

In [64]:
final_df

,_type,url,date,content,renderedContent,id,user,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Search_word,number,text,sentiment_BERT
0,snscrape.modules.twitter.Tweet,https://twitter.com/C_i751/status/1475775243226656773,2021-12-28 10:26:51+00:00,@Roro1122333 @Zatca_sa فارق لا ادعس على وجهك,@Roro1122333 @Zatca_sa فارق لا ادعس على وجهك,1.480000e+18,"{'_type': 'snscrape.modules.twitter.User', 'username': 'C_i751', 'id': 1390574559829663744, 'displayname': '《 #السعودية_للسعوديين》✪', 'description': '🇸🇦 ⬛⬛\nالسعوديه ليست جمعيه خيريه للأجانب ومخلفات الشعوب ، السعوديه ليست وكالة توظيف للشرق والغرب', 'rawDescription': '🇸🇦 ⬛⬛\nالسعوديه ليست جمعيه خيريه للأجانب ومخلفات الشعوب ، السعوديه ليست وكالة توظيف للشرق والغرب', 'descriptionUrls': None, 'verified': False, 'created': '2021-05-07T07:50:05+00:00', 'followersCount': 1900, 'friendsCount': 620, 'statusesCount': 8534, 'favouritesCount': 7227, 'listedCount': 1, 'mediaCount': 1241, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1442419086164627458/IsYrIOmW_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1390574559829663744/1632455766', 'label': None, 'url': 'https://twitter.com/C_i751'}",1.480000e+18,"{'_type': 'snscrape.modules.twitter.User', 'username': 'Roro1122333', 'id': 1490126982, 'displayname': 'بامبو', 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/Roro1122333'}","[{'_type': 'snscrape.modules.twitter.User', 'username': 'Roro1122333', 'id': 1490126982, 'displayname': 'بامبو', 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/Roro1122333'}, {'_type': 'snscrape.modules.twitter.User', 'username': 'Zatca_sa', 'id': 2908767834, 'displayname': 'هيئة الزكاة والضريبة والجمارك', 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/Zatca_sa'}]",NaN,NaN,NaN,NaN,@zatca_sa,NaN,[مستخدم] [مستخدم] فارق لا ادعس على وجهك,1
1,snscrape.modules.twitter.Tweet,https://twitter.com/realEstates_10/status/1487885075001876480,2022-01-30 20:27:00+00:00,⬇️\nالتأثير على التاجر او المطور\n\n لانه سوف تزيد عليهم تكاليف ضريبة\n\nالتصرفات العقارية، و ضريبة الأراضي\n\n البيضاء و تأمين المباني السكنية .,⬇️\nالتأثير على التاجر او المطور\n\n لانه سوف تزيد عليهم تكاليف ضريبة\n\nالتصرفات العقارية، و ضريبة الأراضي\n\n البيضاء و تأمين المباني السكنية .,1.490000e+18,"{'_type': 'snscrape.modules.twitter.User', 'username': 'realEstates_10', 'id': 1277042889416151046, 'displayname': 'عبدالله بن عبدالعزيز', 'description': 'مهتم بالشأن العقاري | مسوق عقاري موثق لدى هيئة العقار برقم 15134 | أعلانات عقارية |\nرسالتي : #تسويق أسهل لعقارك™ أتشرف بمتابعتك🇸🇦', 'rawDescription': 'مهتم بالشأن العقاري | مسوق عقاري موثق لدى هيئة العقار برقم 15134 | أعلانات عقارية |\nرسالتي : #تسويق أسهل لعقارك™ أتشرف بمتابعتك🇸🇦', 'descriptionUrls': None, 'verified': False, 'created': '2020-06-28T00:55:59+00:00', 'followersCount': 2135, 'friendsCount': 306, 'status

In [65]:
final_df.to_excel('BERT_tweets_RESULTS_9_2.xlsx')

In [72]:
len(df_pred)

4698

In [73]:
len(final_df)

4698

In [74]:
final_pred = pd.merge(final_df,df_pred,how="inner" ,left_index=True, right_index=True)

In [77]:
final_pred.to_excel('BERT_tweets_RESULTS_9_2.xlsx')